In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('uucf.csv')
data=data.fillna('')
data.head()

,UserId,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),...,8467: Dumb & Dumber (1994),8587: The Lion King (1994),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003),36955: True Lies (1994)
0,1648,,,,,4,3,,,,...,,4,,,5,3.5,3,,3.5,
1,5136,4.5,5,5,4,5,5,5,3,,...,1,5,,,,5,5,4.5,4,
2,918,5,5,4.5,,3,,5,,5,...,,5,,,,3.5,,,,
3,2824,4.5,,5,,4.5,4,,,5,...,,3.5,,,,,,,,
4,3867,4,4,4.5,,4,3,,,,...,1,4,,,,3,4,4,3.5,3


# Convert given data into a format which can be used for Collaborative Filtering Algorithms.

In [3]:
flat_data = pd.DataFrame(columns=['user','item','rating'])

cols = data.columns
users = data['UserId']
for u in users:
    for c in cols:
        if c =='UserId':
            continue
        i = c.split(':')[0]
        r = data[data['UserId']==u][c].tolist()[0]
        
        #print ('Adding user (%s), item (%s), rating (%s)'%(u,i,r))
        
        #if there are no ratings for a given user and movie then ignore that record
        if r=='':
            continue
        else:
            #put records in seperate dataframe    
            flat_data = flat_data.append({'user': u,'item':i,'rating':r}, ignore_index=True)
        

# Collaboraive Filtering

    - I have used Surprise library for implementing CF see https://surprise.readthedocs.io/en/stable/index.html
    - For a list of supported algorithms see https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

## UUCF Using SVD - Build Collaborative Filtering model Using SVD

    - Model-based Collaborative Filtering: Singular Value Decomposition
    
    - load the required surprise library classes
    - Run a grid search to identify the best hyper parameters for SVD algorithm

In [30]:
from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# The Reader class is used to parse a file containing ratings.
reader = Reader(rating_scale=(1, 5.0))

# The columns must correspond to user id, item id and ratings (in that order).
data_srp_cf = Dataset.load_from_df(flat_data[['user', 'item', 'rating']], reader)

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data_srp_cf)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

trainset = data_srp_cf.build_full_trainset()

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()

0.9461535461111421
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


## Create final SVD model using the identified best parameters

    - SVD is a type of Matrix Factorization Algorithm, https://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [32]:
svd_algo = SVD(n_epochs=10, lr_all=0.005, reg_all= 0.4)
svd_algo.fit(trainset)
predictions_svd = svd_algo.test(testset)

## Make recommendations

    - Here we do two things
    - 1. Predict the ratings which a user would make for a particular movie
    - 2. Recommend movies which the user may want to see basis their Predicted ratings

In [33]:
def predictions_and_recommendations(preds,user,num_recommendation):
    recommended_item = []
    recommended_item_rating = []
    
    # scan through the predictions set and extract items which can be recommended to this user.
    for pred in preds:
        if pred[0]==user:
            recommended_item.append(pred[1])
            recommended_item_rating.append(pred[3])
               
    # create a DF
    recommendations_df = pd.DataFrame(data=list(zip(recommended_item, recommended_item_rating)),columns=['Items','Ratings'])
    recommendations_df = recommendations_df.sort_values(by=['Ratings'], ascending=False)[0:num_recommendation]
    return recommendations_df

In [34]:
pred_recommendations_svd = predictions_and_recommendations(predictions_svd, 89,5)
print ( pred_recommendations_svd)

   Items   Ratings
50    77  4.424460
39  1422  4.405572
27   274  4.404868
19    85  4.358290
5    424  4.331623


## UUCF Using KNNWithMeans

In [35]:
from surprise import KNNWithMeans

# here instead of cosine we could also choose msd, pearson. See https://surprise.readthedocs.io/en/stable/similarities.html for options
knn_algo = KNNWithMeans(k=30, sim={'name': 'cosine','user_based': True})

knn_algo.fit(trainset)
predictions_knn = knn_algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [36]:
pred_recommendations_knn = predictions_and_recommendations(predictions_knn, 89,5)
print ( pred_recommendations_knn)

   Items   Ratings
19    85  4.941163
50    77  4.886317
5    424  4.839573
35   807  4.822805
7    568  4.789277
